In [ ]:
import numpy as np
import scipy.stats as sps
import scipy.optimize as spo
import matplotlib.pyplot as plt
import corner
import sqlite3

In [ ]:
from justice import simulate as sim
from justice import summarize as summ
from justice import visualize as vis
from justice import xform
from justice import lightcurve
from justice.integratedfilters import lsst_filter_integrals

In [ ]:
path_to_db = "data/training_data.db"
conn = sqlite3.connect(path_to_db)

In [ ]:
lcs = lightcurve.PlasticcDatasetLC.get_lc_by_target(conn, 42)

In [ ]:
c = conn.cursor()

In [ ]:
filternames = 'ugrizy'

filterdata = {}
for i in filternames:
    filterdata[i] = np.loadtxt('data/total_'+i+'.dat').T
    
lsst_filter_integrals = {}
for i in filternames:
    lsst_filter_integrals[i] = np.trapz(filterdata[i][1], filterdata[i][0])
    plt.plot(filterdata[i][0], filterdata[i][1])
plt.show()
    
for i in filternames:
    lambda_ends = np.nonzero(filterdata[i][1])[0]
#     print(lambda_ends)
    lsst_filter_integrals[i] = np.trapz(filterdata[i][1], filterdata[i][0]) / float(filterdata[i][0][lambda_ends[-1]] - filterdata[i][0][lambda_ends[0]])
    plt.plot(filterdata[i][0], np.divide(filterdata[i][1], lsst_filter_integrals[i]))
plt.show()
    
# with open('justice/integratedfilters.py', 'w') as filterfile:
#     filterfile.write("# from https://github.com/lsst/throughputs/tree/master/baseline \n")
#     filterfile.write("lsst_filter_integrals = {}".format(tfilterintegrals))

In [ ]:
fig = vis.plot_lcs(lcs[8])

In [ ]:
for j in [lcs[8]]:
    for i in 'ugrizy':
        np.divide(j.bands[i].flux, lsst_filter_integrals[i], out=j.bands[i].flux)
        np.divide(j.bands[i].flux_err, lsst_filter_integrals[i], out=j.bands[i].flux_err)

In [ ]:
fig = vis.plot_lcs(lcs[8])